<a href="https://colab.research.google.com/github/xihajun/Keep-learning/blob/master/code/ruler_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import xml.etree.ElementTree as ElementTree

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
os.chdir("./drive/My Drive/")

In [0]:
os.chdir("./footwear_project")

In [0]:
import numpy as np
import pandas as pd

import os

filelist = []
for dirname, _, filenames in os.walk('./labels'):
    for filename in filenames:
        filelist.append(os.path.join(dirname,filename))
    break

In [0]:
def output_filename_location(filename):
    tree = ElementTree.parse(filename)

    root = tree.getroot()

    for element in root:
        if element.tag == 'filename':
            file_name = element.text
            
        if element.tag == 'object':
            for sub_ele in element:
                if sub_ele.tag == 'bndbox':
                    bndbox = sub_ele
    box = [int(ele.text) for ele in bndbox]
    return file_name, box

In [0]:
training_set = list(map(output_filename_location,filelist))

In [12]:
training_set[0][1][1]

1132

In [0]:
imglist = []
for dirname, _, filenames in os.walk('./images'):
    for filename in filenames:
        imglist.append(os.path.join(dirname,filename))
    break

In [0]:
imglist = imglist[1:] # remove readme

In [0]:
import matplotlib.pyplot as plt

def readimg(filename):
  
  img = plt.imread(filename)
  
  return img.shape

In [0]:
info = list(map(readimg,imglist))
# it shows that images size are the same

In [0]:
(width,height,depth) = info[0]

In [0]:
# task 1 test if that big size is good for autoencoder

In [0]:
# task1.1 generate bw_img for testing

It is not nessasary to get image but just a np.array is enough

In [20]:
training_set[0][1]
# <xmin>812</xmin>
# <ymin>1356</ymin>
# <xmax>3484</xmax>
# <ymax>2332</ymax>

[788, 1132, 3667, 2346]

In [0]:
from scipy.sparse import csr_matrix, hstack, vstack


In [0]:
size = training_set[0][1]

In [23]:
size

[788, 1132, 3667, 2346]

In [0]:
def label_img(training_set):
  matrix = []
  size = training_set[1]
  for i in range(0,3024):
    if i<size[1] or i >size[3]:
      for j in range(0,4032):
        matrix.append(0)
    else:
      for j in range(0,4032):
        if j<size[0] or j>size[2]:
          matrix.append(0)
        else:
          matrix.append(1)
  # load the training img
  img = plt.imread('./images/'+training_set[0])
  img = np.array(img.reshape((width,height,depth))) / 255

  return img,np.array(matrix).reshape(3024,4032)

In [0]:
training = list(map(label_img,training_set))

In [26]:
import tensorflow as tf
from keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, UpSampling2D, Cropping2D
from keras.layers.normalization import BatchNormalization

Using TensorFlow backend.


In [27]:
model = Sequential()

model.add(Conv2D(32,kernel_size = (2,2), 
                input_shape = (width,height,depth), 
                activation = 'relu',
                padding = 'same'))
model.add(MaxPooling2D(pool_size = (2,2), padding = 'same'))

model.add(Conv2D(32,
                kernel_size = (2,2), 
                activation = 'relu',
                padding = 'same'))
model.add(MaxPooling2D(pool_size = (2,2), padding = 'same'))

model.add(Conv2D(1,
                kernel_size = (2,2), 
                activation = 'relu',
                padding = 'same'))
model.add(MaxPooling2D(pool_size = (2,2), padding = 'same'))

model.add(UpSampling2D(size=(2, 2),
    data_format=None,
    interpolation='nearest'))

model.add(UpSampling2D(size=(2, 2),
    data_format=None,
    interpolation='nearest'))

model.add(UpSampling2D(size=(2, 2),
    data_format=None,
    interpolation='nearest'))

In [31]:
model.compile(optimizer='adam', loss='binary_crossentropy')



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
x = [i[0] for i in training]
y = [i[1] for i in training]

In [33]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 3024, 4032, 32)    416       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 1512, 2016, 32)    0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1512, 2016, 32)    4128      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 756, 1008, 32)     0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 756, 1008, 1)      129       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 378, 504, 1)       0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 756, 1008, 1)     

In [27]:
np.array(y).reshape(1,6,3024,4032)

array([[[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
    

In [0]:
model.fit(np.array(x),np.array(y).reshape(6,3024,4032,1),epochs=20)




Epoch 1/20







In [0]:
(width,height,depth) = info[0]

In [29]:
(width,height,depth)

(3024, 4032, 3)

In [30]:
x[0]

NameError: ignored

In [0]:
from skimage import transform,data
dst=transform.resize(img, (378,504))

In [0]:
# task 2 resize it into 300,400